In [1]:
#imports
import pandas as pd
import numpy as np
import os
import psycopg2
from decimal import Decimal
from dotenv import load_dotenv
from functions import sql_to_dataframe, connect, fetch_column_names
load_dotenv()
from sqlalchemy import create_engine, inspect
import matplotlib.pyplot as plt

from importlib import reload

from lib import masking_functions, relative_entropy_copy, relative_entropy_estimator
reload(masking_functions)
reload(relative_entropy_copy)
reload(relative_entropy_estimator)

from lib.masking_functions import mask, noise_date, generalize_date, generalize_number, suppress, bucketize, inverse_generalize_date_SO, inverse_generalize_date, inverse_bucketize_SO, add_relative_noise, inverse_bucketize, FloatAlphabet, DateAlphabet, inverse_blur_string, inverse_bucketize, inverse_generalize_number, blur_string, inverse_suppress_phone, inverse_blur_phone, inverse_suppress_segment, inverse_generalize_number_SO, inverse_noise_date, inverse_noise_date_SO
from lib.relative_entropy_copy import compute_prob_dist, specification_prob, relative_entropy
from lib.relative_entropy_estimator import DB_stats, estimate_rel_entropy, convert_to_array, get_stat

In [2]:
from datetime import datetime, timedelta

def generate_date_range(start_date_str, end_date_str):
    """
    Generate all dates between two given dates (inclusive).

    Args:
        start_date_str (str): The start date in "YYYY-MM-DD" format.
        end_date_str (str): The end date in "YYYY-MM-DD" format.

    Returns:
        list: A list of dates as strings in "YYYY-MM-DD" format.
    """
    try:
        # Convert string dates to datetime objects
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
        end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

        # Check if the start date is before or the same as the end date
        if start_date > end_date:
            raise ValueError("The start date must be before or the same as the end date.")

        # Generate the list of dates
        current_date = start_date
        date_list = []
        while current_date <= end_date:
            date_list.append(current_date.strftime("%Y-%m-%d"))
            current_date += timedelta(days=1)  # Move to the next day

        return date_list
    
    except ValueError as e:
        raise ValueError(f"Date error: {e}")

# Example usage
start_date = "1990-01-01"
end_date = "2000-01-01"
all_dates = generate_date_range(start_date, end_date)

# DB connection

In [3]:
# Load environment variables from .env file
load_dotenv(dotenv_path='/Users/avanedaei/Desktop/thesis/main/thesis/touch.env')

True

## Opening Connection

In [4]:
# Fetch connection details from environment variables
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')

db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

try:
    engine = create_engine(db_url)
    connection = engine.connect()
    print("Connection successful")
except Exception as e:
    print(f"Error connecting to the database: {e}")

Connection successful


## Customer Dataset

In [5]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_original = pd.read_sql(query, connection)
    if customer_original.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_original.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,945,Customer#000000945,300zKNJ9lg,15,25-542-662-1673,9615.0,BUILDING,. ironic deposits haggle among the carefully r...
1,1171,Customer#000001171,GatOC LsLU9MkgyaNMYH,8,18-457-394-2863,7659.0,HOUSEHOLD,c dolphins. accounts are slyl
2,1356,Customer#000001356,3SLzAiW4PihnFUE243 AHKkwtL1PCj,5,15-656-712-5740,927.0,HOUSEHOLD,fully pending deposits. carefully unusual acco...
3,1513,Customer#000001513,CkEgq3Yvj9kGkHvVeUELT1UP9HBnHwiEIFzRWNTA,10,20-670-367-4252,8434.0,FURNITURE,y regular accounts cajole blithely
4,1594,Customer#000001594,8No1IYGij7,13,23-416-484-3099,4797.0,FURNITURE,final packages wake idly. quickly regular pack


In [6]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p3'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p3_masked = pd.read_sql(query, connection)
    if customer_p3_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p3_masked.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,945,Customer#000000945,300zKNJ9lg,15,25-542-662-1673,9600.0,BUILDING,. ironic deposits haggle among the carefully r...
1,1171,Customer#000001171,GatOC LsLU9MkgyaNMYH,8,18-457-394-2863,7600.0,HOUSEHOLD,c dolphins. accounts are slyl
2,1356,Customer#000001356,3SLzAiW4PihnFUE243 AHKkwtL1PCj,5,15-656-712-5740,900.0,HOUSEHOLD,fully pending deposits. carefully unusual acco...
3,1513,Customer#000001513,CkEgq3Yvj9kGkHvVeUELT1UP9HBnHwiEIFzRWNTA,10,20-670-367-4252,8400.0,FURNITURE,y regular accounts cajole blithely
4,1594,Customer#000001594,8No1IYGij7,13,23-416-484-3099,4700.0,FURNITURE,final packages wake idly. quickly regular pack


In [7]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p1'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p1_masked = pd.read_sql(query, connection)
    if customer_p1_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p1_masked.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,945,Customer#000000945,300zKNJ9lg,15,25-542-662-1673,9615.0,XXXXX,. ironic deposits haggle among the carefully r...
1,1171,Customer#000001171,GatOC LsLU9MkgyaNMYH,8,18-457-394-2863,7659.0,XXXXX,c dolphins. accounts are slyl
2,1356,Customer#000001356,3SLzAiW4PihnFUE243 AHKkwtL1PCj,5,15-656-712-5740,927.0,XXXXX,fully pending deposits. carefully unusual acco...
3,1513,Customer#000001513,CkEgq3Yvj9kGkHvVeUELT1UP9HBnHwiEIFzRWNTA,10,20-670-367-4252,8434.0,XXXXX,y regular accounts cajole blithely
4,1594,Customer#000001594,8No1IYGij7,13,23-416-484-3099,4797.0,XXXXX,final packages wake idly. quickly regular pack


In [8]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p2'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p2_masked = pd.read_sql(query, connection)
    if customer_p2_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p2_masked.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,945,Customer#000000945,300zKNJ9lg,15,25-542-662-1673,9500.0,BUILDING,. ironic deposits haggle among the carefully r...
1,1171,Customer#000001171,GatOC LsLU9MkgyaNMYH,8,18-457-394-2863,7500.0,HOUSEHOLD,c dolphins. accounts are slyl
2,1356,Customer#000001356,3SLzAiW4PihnFUE243 AHKkwtL1PCj,5,15-656-712-5740,750.0,HOUSEHOLD,fully pending deposits. carefully unusual acco...
3,1513,Customer#000001513,CkEgq3Yvj9kGkHvVeUELT1UP9HBnHwiEIFzRWNTA,10,20-670-367-4252,8250.0,FURNITURE,y regular accounts cajole blithely
4,1594,Customer#000001594,8No1IYGij7,13,23-416-484-3099,4750.0,FURNITURE,final packages wake idly. quickly regular pack


In [9]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_stats_original = pd.read_sql(query, connection)
    if customer_stats_original.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_stats_original.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_acctbal,10839.0000,"[1348, 1512, 7118, 846, 1251, 2493, 2564, 2979...","[0.00033333333, 0.00033333333, 0.00033333333, ...","[-1000, -883, -775, -665, -560, -442, -334, -2..."
1,c_comment,-0.9997,None,None,[ Tiresias affix ideas. carefully special acco...
2,c_custkey,-1.0000,None,None,"[2, 1538, 2979, 4422, 5958, 7557, 9104, 10609,..."
3,c_address,-1.0000,None,None,"[ 1Ur sAbsaAHmBcIelil9U85KIXmyQkdyT,qop, dNq..."
4,c_nationkey,25.0000,"[6, 19, 17, 12, 13, 23, 18, 9, 16, 1, 20, 15, ...","[0.041366667, 0.0413, 0.0411, 0.040866666, 0.0...",None
5,c_name,-1.0000,None,None,"[Customer#000000002, Customer#000001538, Custo..."


In [10]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p3_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p3_stats_masked = pd.read_sql(query, connection)
    if customer_p3_stats_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p3_stats_masked.head(9)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,-1.0000,None,None,"[4, 1505, 3048, 4605, 6245, 7625, 9118, 10572,..."
1,c_address,-1.0000,None,None,"[ 5Ftcw1tnQoDQ62he1J2Xn93oPjIi, cCO9MiiVgYtV..."
2,c_nationkey,25.0000,"[10, 9, 8, 6, 19, 3, 2, 13, 17, 15, 7, 4, 14, ...","[0.041966666, 0.041833334, 0.0417, 0.0414, 0.0...",None
3,c_mktsegment,5.0000,"[BUILDING , MACHINERY , HOUSEHOLD , AUTOMOBIL...","[0.20333333, 0.1996, 0.19936667, 0.1993, 0.1984]",None
4,c_comment,-0.9994,None,None,[ Tiresias affix ideas. carefully special acco...
5,c_name,-1.0000,None,None,"[Customer#000000004, Customer#000001505, Custo..."
6,c_phone,-1.0000,None,None,"[10-100-106-1617, 10-321-134-7311, 10-570-750-..."
7,c_acctbal,111.0000,"[6600, 2500, -400]","[0.010333333, 0.010266666, 0.0101666665]","[-1000, -900, -800, -700, -600, -500, -300, -2..."


In [11]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p1_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p1_stats_masked = pd.read_sql(query, connection)
    if customer_p1_stats_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p1_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,-1.0,None,None,"[2, 1491, 2971, 4453, 5967, 7371, 8834, 10365,..."
1,c_name,-1.0,None,None,"[Customer#000000002, Customer#000001491, Custo..."
2,c_address,-1.0,None,None,"[ 2uZwVhQvwA, ZslQM6KHkJhm6LrB64jZsgxqBd Cj..."
3,c_phone,-1.0,None,None,"[10-100-220-4520, 10-309-224-6978, 10-538-988-..."
4,c_acctbal,10873.0,"[4300, 5633, -971, 2621, 5274, 6387, 9853, -18...","[0.00046666668, 0.00036666667, 0.00033333333, ...","[-1000, -888, -785, -674, -559, -452, -353, -2..."
5,c_nationkey,25.0,"[6, 21, 7, 13, 14, 19, 10, 15, 20, 22, 9, 8, 1...","[0.0413, 0.041033335, 0.04083333, 0.0408, 0.04...",None


In [12]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p2_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p2_stats_masked = pd.read_sql(query, connection)
    if customer_p2_stats_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p2_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,-1.0,None,None,"[4, 1572, 3151, 4717, 6333, 7709, 9406, 11005,..."
1,c_name,-1.0,None,None,"[Customer#000000004, Customer#000001572, Custo..."
2,c_address,-1.0,None,None,"[ 4WsGuqIsPAG4qAEjj32S9d, bLU,snFrnRBKixCHZj..."
3,c_nationkey,25.0,"[8, 14, 0, 20, 19, 13, 16, 9, 1, 2, 3, 11, 15,...","[0.042433333, 0.041433334, 0.0409, 0.0409, 0.0...",None
4,c_phone,-1.0,None,None,"[10-100-864-5796, 10-326-985-6158, 10-544-687-..."
5,c_acctbal,45.0,"[9750, 2750, 5500, 1750, 2000, 3250, 6500, 400...","[0.024533333, 0.024166666, 0.0239, 0.023866666...",None


In [13]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p4'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p4_masked = pd.read_sql(query, connection)
    if customer_p4_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p4_masked.head(6)

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,945,Customer#000000945,300zKNJ9lg,15,25-542-662-1XXX,9615.0,BUILDING,. ironic deposits haggle among the carefully r...
1,1171,Customer#000001171,GatOC LsLU9MkgyaNMYH,8,18-457-394-2XXX,7659.0,HOUSEHOLD,c dolphins. accounts are slyl
2,1356,Customer#000001356,3SLzAiW4PihnFUE243 AHKkwtL1PCj,5,15-656-712-5XXX,927.0,HOUSEHOLD,fully pending deposits. carefully unusual acco...
3,1513,Customer#000001513,CkEgq3Yvj9kGkHvVeUELT1UP9HBnHwiEIFzRWNTA,10,20-670-367-4XXX,8434.0,FURNITURE,y regular accounts cajole blithely
4,1594,Customer#000001594,8No1IYGij7,13,23-416-484-3XXX,4797.0,FURNITURE,final packages wake idly. quickly regular pack
5,1634,Customer#000001634,mTRMQ9143TTe5kHsD2FdNE7proZ,24,34-186-980-9XXX,4030.0,FURNITURE,s are thinly fluffily ironic requests. pinto b...


In [14]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p4_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p4_stats_masked = pd.read_sql(query, connection)
    if customer_p4_stats_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p4_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,-1.0000,None,None,"[12, 1548, 3113, 4495, 5975, 7460, 8920, 10373..."
1,c_name,-1.0000,None,None,"[Customer#000000012, Customer#000001548, Custo..."
2,c_address,-1.0000,None,None,"[ 1Ur sAbsaAHmBcIelil9U85KIXmyQkdyT,qop, bag..."
3,c_nationkey,25.0000,"[9, 4, 10, 22, 23, 3, 15, 11, 5, 6, 18, 20, 8,...","[0.041866668, 0.041666668, 0.0414, 0.041333333...",None
4,c_phone,-0.9991,None,None,"[10-100-301-2XXX, 10-334-644-7XXX, 10-593-484-..."
5,c_mktsegment,5.0000,"[BUILDING , AUTOMOBILE, MACHINERY , FURNITURE...","[0.20416667, 0.20043333, 0.19983333, 0.1994666...",None


## Order Dataset

In [15]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_original = pd.read_sql(query, connection)
    if orders_original.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_original.head()

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,1,36901,O,173665.47,1996-01-02,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,2,78002,O,46929.18,1996-12-01,1-URGENT,Clerk#000000880,0,"foxes. pending accounts at the pending, silen..."
2,3,123314,F,193846.25,1993-10-14,5-LOW,Clerk#000000955,0,sly final accounts boost. carefully regular id...
3,4,136777,O,32151.78,1995-10-11,5-LOW,Clerk#000000124,0,"sits. slyly regular warthogs cajole. regular, ..."
4,5,44485,F,144659.20,1994-07-30,5-LOW,Clerk#000000925,0,quickly. bold deposits sleep slyly. packages u...


In [16]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_p2'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_masked_p2 = pd.read_sql(query, connection)
    if orders_masked_p2.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_masked_p2.head()

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,1,36901,O,173665.47,1996-01-01,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,2,78002,O,46929.18,1996-12-01,1-URGENT,Clerk#000000880,0,"foxes. pending accounts at the pending, silen..."
2,3,123314,F,193846.25,1993-10-01,5-LOW,Clerk#000000955,0,sly final accounts boost. carefully regular id...
3,4,136777,O,32151.78,1995-10-01,5-LOW,Clerk#000000124,0,"sits. slyly regular warthogs cajole. regular, ..."
4,5,44485,F,144659.20,1994-07-01,5-LOW,Clerk#000000925,0,quickly. bold deposits sleep slyly. packages u...


In [17]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_p1'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_masked_p1 = pd.read_sql(query, connection)
    if orders_masked_p1.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_masked_p1.head()

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,1,36901,O,173665.47,1995-12-23,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,2,78002,O,46929.18,1996-11-25,1-URGENT,Clerk#000000880,0,"foxes. pending accounts at the pending, silen..."
2,3,123314,F,193846.25,1993-10-06,5-LOW,Clerk#000000955,0,sly final accounts boost. carefully regular id...
3,4,136777,O,32151.78,1995-10-20,5-LOW,Clerk#000000124,0,"sits. slyly regular warthogs cajole. regular, ..."
4,5,44485,F,144659.20,1994-07-27,5-LOW,Clerk#000000925,0,quickly. bold deposits sleep slyly. packages u...


In [18]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_stats_original = pd.read_sql(query, connection)
    if orders_stats_original.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_stats_original.head()

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,o_orderkey,-1.000000,None,None,"[34, 57284, 117574, 177600, 234790, 291392, 34..."
1,o_custkey,85643.000000,None,None,"[16, 1468, 2912, 4394, 5839, 7450, 8887, 10399..."
2,o_orderstatus,3.000000,"[O, F, P]","[0.48843333, 0.48576668, 0.0258]",None
3,o_totalprice,-0.961894,None,None,"[927.91, 5838.56, 10936.78, 15365.87, 19403.67..."
4,o_orderdate,2406.000000,"[1997-01-09, 1992-04-30, 1997-07-28, 1997-10-0...","[0.00086666667, 0.00083333335, 0.00083333335, ...","[1992-01-01, 1992-01-26, 1992-02-20, 1992-03-1..."


In [19]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_p2_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_stats_masked_p2 = pd.read_sql(query, connection)
    if orders_stats_masked_p2.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_stats_masked_p2.head()

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,o_orderkey,-1.000000,None,None,"[324, 59395, 115174, 173478, 241733, 299364, 3..."
1,o_custkey,87717.000000,None,None,"[37, 1582, 3130, 4588, 6109, 7630, 9316, 10690..."
2,o_orderstatus,3.000000,"[F, O, P]","[0.4883, 0.4865, 0.0252]",None
3,o_totalprice,-0.952812,None,None,"[952.90, 5646.13, 10082.51, 14335.16, 18547.70..."
4,o_orderdate,80.000000,"[1998-07-01, 1994-03-01, 1994-01-01, 1997-05-0...","[0.014366667, 0.013966667, 0.013933334, 0.0138...",None


In [20]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_p1_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_stats_masked_p1 = pd.read_sql(query, connection)
    if orders_stats_masked_p1.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_stats_masked_p1.head()

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,o_orderkey,-1.000000,None,None,"[36, 58371, 116224, 179558, 239268, 299910, 36..."
1,o_custkey,85910.000000,None,None,"[8, 1591, 2938, 4447, 5983, 7547, 9058, 10610,..."
2,o_orderstatus,3.000000,"[O, F, P]","[0.49116668, 0.48346666, 0.025366666]",None
3,o_totalprice,-0.946851,None,None,"[884.52, 5557.24, 9959.78, 14203.77, 18033.36,..."
4,o_orderdate,2423.000000,"[1996-03-14, 1996-11-30, 1993-12-17, 1998-07-0...","[0.0009, 0.0009, 0.00086666667, 0.00086666667,...","[1991-12-22, 1992-01-25, 1992-02-17, 1992-03-1..."


## Closing Connection

In [21]:
#closing the connection
try:
    connection.close()
    print("Connection closed")
except Exception as e:
    print(f"Error closing the connection: {e}")
# Let’s see if we loaded the df successfully

Connection closed


### Masking Customer Table

#### suppressing the c_mktsegment column

In [22]:
# clean up spaces from column c_mktsegment
df['c_mktsegment'] = df['c_mktsegment'].str.replace(r'\s+', '', regex=True)

NameError: name 'df' is not defined

In [ ]:
customer_m1 = mask(df, suppress, 'c_mktsegment')

In [ ]:
customer_m1.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m1.tbl.csv", sep=',', header=False, index=False)
customer_m1.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m1.csv", sep=',', header=False, index=False)

#### generalizing the c_acctbal column number

In [ ]:

customer_m2 = mask(df, generalize_number, 'c_acctbal', 100)
customer_m2.head()

In [ ]:
customer_m2.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m2.csv", sep=',', header=False, index=False)
customer_m2.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m2.tbl.csv", sep=',', header=False, index=False)

#### bucketizing the c_acctbal column number

In [ ]:
customer_m3 = mask(df, bucketize, 'c_acctbal', 500)

In [ ]:
customer_m3.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m3.csv", sep=',', header=False, index=False)
customer_m3.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m3.tbl.csv", sep=',', header=False, index=False)

In [ ]:
customer_m4 = mask(df, bucketize, 'c_acctbal', 500)

#### generalizing order date based on month

In [ ]:
orders_m2 = mask(orders_original, generalize_date, 'o_orderdate', "MONTH")

In [ ]:
orders_m2.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/orders_m2.tbl.csv", sep=',', header=False, index=False)
orders_m2.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/orders_m2.csv", sep=',', header=False, index=False)

## customer table

### With Masking Functions

##### example: estimating entropy c_mktsegment

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment'])
q = compute_prob_dist(customer_m1, ['c_mktsegment'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_blur_string)
#print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

##### example: estimating entropy c_acctbal

In [ ]:
p = compute_prob_dist(df, ['c_acctbal'])
q = compute_prob_dist(customer_m2, ['c_acctbal'])

# Convert probability distribution reverting the masking function
# print(q)
divisor = 100
new_q = specification_prob(q,inverse_generalize_number(divisor))
#print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

In [ ]:
# p = compute_prob_dist(df, ['c_acctbal'])
# q = compute_prob_dist(customer_m3, ['c_acctbal'])

# # Convert probability distribution reverting the masking function
# # print(q)
# new_q = specification_prob(q,inverse_bucketize(['c_acctbal'],0.01))
# #print(new_q)
# row, col = df.shape
# size = row * col
# relative_entropy(p, new_q,size)

### With Materialized Views

#### entropy over single column

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment'])
q = compute_prob_dist(materialized_view, ['c_mktsegment'])

# Convert probability distribution reverting the masking function
new_q = specification_prob(q,inverse_suppress_segment)
print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)


#### entropy over multiple columns

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment', 'c_nationkey'])
q = compute_prob_dist(materialized_view, ['c_mktsegment', 'c_nationkey'])

# Convert probability distribution reverting the masking function
new_q = specification_prob(q,inverse_suppress_segment)
print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment', 'c_comment'])
q = compute_prob_dist(materialized_view, ['c_mktsegment', 'c_comment'])

# Convert probability distribution reverting the masking function
new_q = specification_prob(q,inverse_suppress_segment)
print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

# Estimating Relative Entropy

## customer table

### c_p4 - blurred c_phone

In [ ]:
size = 6001215
# df.head()
customer_stats_original.fillna(np.NaN, inplace=True)
customer_p4_stats_masked.fillna(np.NaN, inplace=True)
# print(customer_stats_original.head())

customer_stats_original["n_distinct"] = customer_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
customer_p4_stats_masked["n_distinct"] = customer_p2_stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [ ]:
p = compute_prob_dist(customer_original, ['c_phone'])
q = compute_prob_dist(customer_p4_masked, ['c_phone'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_blur_phone)
#print(new_q)
# row, col = customer_original.shape
# size = row * col
# relative_entropy(p, new_q,size)

: 

### c_p2 - generalizing c_acctbal - 250

In [ ]:
size = 6001215
# df.head()
customer_stats_original.fillna(np.NaN, inplace=True)
customer_p2_stats_masked.fillna(np.NaN, inplace=True)
# print(customer_stats_original.head())

customer_stats_original["n_distinct"] = customer_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
customer_p2_stats_masked["n_distinct"] = customer_p2_stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [ ]:
p = compute_prob_dist(customer_original, ['c_acctbal'])
q = compute_prob_dist(customer_p2_masked, ['c_acctbal'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_generalize_number_SO(250))
#print(new_q)
row, col = customer_original.shape
size = row * col
relative_entropy(p, new_q,size)

0.05429290139624589

In [ ]:
customer_stats_original.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_acctbal,10839,"[1348, 1512, 7118, 846, 1251, 2493, 2564, 2979...","[0.00033333333, 0.00033333333, 0.00033333333, ...","[-1000, -883, -775, -665, -560, -442, -334, -2..."
1,c_comment,5999414,NaN,NaN,[ Tiresias affix ideas. carefully special acco...
2,c_custkey,6001215,NaN,NaN,"[2, 1538, 2979, 4422, 5958, 7557, 9104, 10609,..."
3,c_address,6001215,NaN,NaN,"[ 1Ur sAbsaAHmBcIelil9U85KIXmyQkdyT,qop, dNq..."
4,c_nationkey,25,"[6, 19, 17, 12, 13, 23, 18, 9, 16, 1, 20, 15, ...","[0.041366667, 0.0413, 0.0411, 0.040866666, 0.0...",NaN
5,c_name,6001215,NaN,NaN,"[Customer#000000002, Customer#000001538, Custo..."


In [ ]:
customer_p2_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,6001215,NaN,NaN,"[4, 1572, 3151, 4717, 6333, 7709, 9406, 11005,..."
1,c_name,6001215,NaN,NaN,"[Customer#000000004, Customer#000001572, Custo..."
2,c_address,6001215,NaN,NaN,"[ 4WsGuqIsPAG4qAEjj32S9d, bLU,snFrnRBKixCHZj..."
3,c_nationkey,25,"[8, 14, 0, 20, 19, 13, 16, 9, 1, 2, 3, 11, 15,...","[0.042433333, 0.041433334, 0.0409, 0.0409, 0.0...",NaN
4,c_phone,6001215,NaN,NaN,"[10-100-864-5796, 10-326-985-6158, 10-544-687-..."
5,c_acctbal,45,"[9750, 2750, 5500, 1750, 2000, 3250, 6500, 400...","[0.024533333, 0.024166666, 0.0239, 0.023866666...",NaN


In [ ]:
# customer_stats_original.loc[customer_stats_original["attname"] == "c_acctbal", "histogram_bounds"] = customer_stats_original[customer_stats_original["attname"] == "c_acctbal"]["histogram_bounds"].apply(lambda x : [float(i) for i in x])

In [ ]:
customer_stats_original_db = DB_stats(customer_stats_original, "c_acctbal")
customer_p2_stats_masked_db = DB_stats(customer_p2_stats_masked, "c_acctbal")
customer_p2_stats_masked_db.perturbation()
customer_p2_stats_masked_db.inverse_dist(inverse_generalize_number_SO(250,  1, "{:0f}"))

In [ ]:
customer_stats_original_db.histogram_bounds

array(['-1000', '-883', '-775', '-665', '-560', '-442', '-334', '-225',
       '-116', '-10', '100', '214', '329', '437', '551', '656', '767',
       '868', '979', '1094', '1194', '1301', '1411', '1516', '1620',
       '1732', '1846', '1953', '2067', '2178', '2289', '2395', '2507',
       '2626', '2744', '2859', '2974', '3079', '3203', '3313', '3425',
       '3536', '3651', '3762', '3867', '3977', '4079', '4185', '4293',
       '4405', '4521', '4631', '4750', '4864', '4974', '5077', '5186',
       '5287', '5396', '5507', '5607', '5718', '5831', '5934', '6046',
       '6157', '6263', '6373', '6484', '6590', '6688', '6797', '6916',
       '7028', '7130', '7233', '7348', '7463', '7575', '7681', '7795',
       '7911', '8025', '8130', '8240', '8348', '8454', '8565', '8682',
       '8798', '8917', '9019', '9134', '9251', '9359', '9462', '9561',
       '9668', '9781', '9895', '10000'], dtype='<U5')

In [ ]:
customer_p2_stats_masked_db.histogram_bounds

array([], dtype=float64)

In [ ]:
customer_p2_stats_masked_db.inverse_histogram_bounds(inverse_generalize_number_SO(250.0, 0.01, "{:.2f}"))

In [ ]:
customer_p2_stats_masked_db.unmsk_histogram_bounds

array([], dtype=float64)

In [ ]:
min_value = -1000
max_value = 100000
step = 1

c_acctbal_alphabet = []
for number in np.arange(min_value, max_value + step, step):
        # Format the number to two decimal places
        # formatted_number = f"{number:.2f}"
        # Combine the letter with the formatted number
        formatted_number = f"{number}"
        # if formatted_number.endswith('.00'):
        #         formatted_number = formatted_number[:-1]
        c_acctbal_alphabet.append(formatted_number)

In [ ]:
len(c_acctbal_alphabet)
c_acctbal_alphabet[:6]
print(c_acctbal_alphabet[:6])

['-1000', '-999', '-998', '-997', '-996', '-995']


In [ ]:
len(c_acctbal_alphabet)
print(c_acctbal_alphabet[100000:100005])

['99000', '99001', '99002', '99003', '99004']


In [ ]:
customer_stats_original_db.estimate_hist_freq(c_acctbal_alphabet)

In [ ]:
freq = 0.0

for i, x in enumerate(customer_stats_original_db.hist_apprx_freq):
    freq += x * customer_stats_original_db.hist_apprx_n_distinct[i]

print(freq)

0.9951000000099985


In [ ]:
customer_p2_stats_masked_db.estimate_hist_freq(c_acctbal_alphabet, True)

In [ ]:
freq = 0.0

for x in customer_p2_stats_masked_db.most_common_freqs:
    freq += x
print(freq)

1.0000000006699998


In [ ]:
entropy_c_generalize_acctbal_p2 = estimate_rel_entropy(customer_stats_original_db, customer_p2_stats_masked_db)
print(entropy_c_generalize_acctbal_p2)

0.060966127772057566


### c_p3 - generalizing c_acctbal - 100

In [ ]:
size = 6001215
# df.head()
customer_stats_original.fillna(np.NaN, inplace=True)
customer_p3_masked.fillna(np.NaN, inplace=True)
# print(customer_stats_original.head())

customer_stats_original["n_distinct"] = customer_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
customer_p3_stats_masked["n_distinct"] = customer_p3_stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [ ]:
p = compute_prob_dist(customer_original, ['c_acctbal'])
q = compute_prob_dist(customer_p3_masked, ['c_acctbal'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_generalize_number_SO(100))
#print(new_q)
row, col = customer_original.shape
size = row * col
relative_entropy(p, new_q,size)

0.054038876325570244

In [ ]:
customer_stats_original.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_acctbal,10839,"[1348, 1512, 7118, 846, 1251, 2493, 2564, 2979...","[0.00033333333, 0.00033333333, 0.00033333333, ...","[-1000, -883, -775, -665, -560, -442, -334, -2..."
1,c_comment,5999414,NaN,NaN,[ Tiresias affix ideas. carefully special acco...
2,c_custkey,6001215,NaN,NaN,"[2, 1538, 2979, 4422, 5958, 7557, 9104, 10609,..."
3,c_address,6001215,NaN,NaN,"[ 1Ur sAbsaAHmBcIelil9U85KIXmyQkdyT,qop, dNq..."
4,c_nationkey,25,"[6, 19, 17, 12, 13, 23, 18, 9, 16, 1, 20, 15, ...","[0.041366667, 0.0413, 0.0411, 0.040866666, 0.0...",NaN
5,c_name,6001215,NaN,NaN,"[Customer#000000002, Customer#000001538, Custo..."


In [ ]:
customer_p3_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,6001215,None,None,"[4, 1505, 3048, 4605, 6245, 7625, 9118, 10572,..."
1,c_address,6001215,None,None,"[ 5Ftcw1tnQoDQ62he1J2Xn93oPjIi, cCO9MiiVgYtV..."
2,c_nationkey,25,"[10, 9, 8, 6, 19, 3, 2, 13, 17, 15, 7, 4, 14, ...","[0.041966666, 0.041833334, 0.0417, 0.0414, 0.0...",None
3,c_mktsegment,5,"[BUILDING , MACHINERY , HOUSEHOLD , AUTOMOBIL...","[0.20333333, 0.1996, 0.19936667, 0.1993, 0.1984]",None
4,c_comment,5997614,None,None,[ Tiresias affix ideas. carefully special acco...
5,c_name,6001215,None,None,"[Customer#000000004, Customer#000001505, Custo..."


In [ ]:
# customer_stats_original.loc[customer_stats_original["attname"] == "c_acctbal", "histogram_bounds"] = customer_stats_original[customer_stats_original["attname"] == "c_acctbal"]["histogram_bounds"].apply(lambda x : [float(i) for i in x])

In [ ]:
customer_stats_original_db = DB_stats(customer_stats_original, "c_acctbal")
customer_p3_stats_masked_db = DB_stats(customer_p3_stats_masked, "c_acctbal")
# customer_p3_stats_masked_db.perturbation()
customer_p3_stats_masked_db.inverse_dist(inverse_generalize_number_SO(100,  1, "{:.0f}"))

In [ ]:
customer_stats_original_db.histogram_bounds

array(['-1000', '-883', '-775', '-665', '-560', '-442', '-334', '-225',
       '-116', '-10', '100', '214', '329', '437', '551', '656', '767',
       '868', '979', '1094', '1194', '1301', '1411', '1516', '1620',
       '1732', '1846', '1953', '2067', '2178', '2289', '2395', '2507',
       '2626', '2744', '2859', '2974', '3079', '3203', '3313', '3425',
       '3536', '3651', '3762', '3867', '3977', '4079', '4185', '4293',
       '4405', '4521', '4631', '4750', '4864', '4974', '5077', '5186',
       '5287', '5396', '5507', '5607', '5718', '5831', '5934', '6046',
       '6157', '6263', '6373', '6484', '6590', '6688', '6797', '6916',
       '7028', '7130', '7233', '7348', '7463', '7575', '7681', '7795',
       '7911', '8025', '8130', '8240', '8348', '8454', '8565', '8682',
       '8798', '8917', '9019', '9134', '9251', '9359', '9462', '9561',
       '9668', '9781', '9895', '10000'], dtype='<U5')

In [ ]:
customer_p3_stats_masked_db.histogram_bounds

array(['-1000', '-900', '-800', '-700', '-600', '-500', '-300', '-200',
       '-100', '0', '100', '200', '300', '500', '600', '700', '800',
       '900', '1000', '1100', '1200', '1300', '1400', '1500', '1600',
       '1700', '1900', '2000', '2100', '2200', '2300', '2400', '2600',
       '2700', '2800', '2900', '3000', '3100', '3200', '3300', '3400',
       '3500', '3600', '3700', '3800', '3900', '4000', '4100', '4300',
       '4300', '4400', '4600', '4700', '4800', '4900', '5000', '5100',
       '5200', '5300', '5400', '5500', '5600', '5700', '5800', '6000',
       '6100', '6200', '6300', '6400', '6500', '6700', '6800', '6900',
       '7000', '7200', '7300', '7400', '7500', '7600', '7700', '7800',
       '7900', '8000', '8100', '8200', '8300', '8400', '8600', '8700',
       '8800', '8900', '9000', '9100', '9200', '9300', '9400', '9500',
       '9600', '9700', '9800', '10000'], dtype='<U5')

In [ ]:
customer_p3_stats_masked_db.inverse_histogram_bounds(inverse_generalize_number_SO(100, 1, "{:.0f}"))

In [ ]:
customer_p3_stats_masked_db.unmsk_histogram_bounds

array(['-1000', '-900', '-800', '-700', '-600', '-500', '-300', '-200',
       '-100', '0', '100', '200', '300', '500', '600', '700', '800',
       '900', '1000', '1100', '1200', '1300', '1400', '1500', '1600',
       '1700', '1900', '2000', '2100', '2200', '2300', '2400', '2600',
       '2700', '2800', '2900', '3000', '3100', '3200', '3300', '3400',
       '3500', '3600', '3700', '3800', '3900', '4000', '4100', '4300',
       '4300', '4400', '4600', '4700', '4800', '4900', '5000', '5100',
       '5200', '5300', '5400', '5500', '5600', '5700', '5800', '6000',
       '6100', '6200', '6300', '6400', '6500', '6700', '6800', '6900',
       '7000', '7200', '7300', '7400', '7500', '7600', '7700', '7800',
       '7900', '8000', '8100', '8200', '8300', '8400', '8600', '8700',
       '8800', '8900', '9000', '9100', '9200', '9300', '9400', '9500',
       '9600', '9700', '9800', '10099'], dtype='<U5')

In [ ]:
min_value = -1000
max_value = 100000
step = 1

c_acctbal_alphabet = []
for number in np.arange(min_value, max_value + step, step):
        # Format the number to two decimal places
        # formatted_number = f"{number:.2f}"
        # Combine the letter with the formatted number
        formatted_number = f"{number}"
        # if formatted_number.endswith('.00'):
        #         formatted_number = formatted_number[:-1]
        c_acctbal_alphabet.append(formatted_number)

In [ ]:
len(c_acctbal_alphabet)
c_acctbal_alphabet[:6]
print(c_acctbal_alphabet[:6])

['-1000', '-999', '-998', '-997', '-996', '-995']


In [ ]:
len(c_acctbal_alphabet)
print(c_acctbal_alphabet[100000:100005])

['99000', '99001', '99002', '99003', '99004']


In [ ]:
customer_stats_original_db.estimate_hist_freq(c_acctbal_alphabet)

In [ ]:
freq = 0.0

for i, x in enumerate(customer_stats_original_db.hist_apprx_freq):
    freq += x * customer_stats_original_db.hist_apprx_n_distinct[i]

print(freq)

0.9951000000099985


In [ ]:
customer_p3_stats_masked_db.estimate_hist_freq(c_acctbal_alphabet, True)

In [ ]:
freq = 0.0

for x in customer_p3_stats_masked_db.most_common_freqs:
    freq += x
print(freq)

0.0307666655


In [ ]:
entropy_c_generalize_acctbal_p2 = estimate_rel_entropy(customer_stats_original_db, customer_p2_stats_masked_db)
print(entropy_c_generalize_acctbal_p2)

0.060966127772057566


### c_mktsegment

In [ ]:
size = 150000
# df.head()
customer_stats_original.fillna(np.NaN, inplace=True)
customer_stats_masked.fillna(np.NaN, inplace=True)
print(customer_stats_masked.head())
customer_stats_original["n_distinct"] = customer_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
customer_stats_masked["n_distinct"] = customer_stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [ ]:
# stats_masked.head()
mktsegment_stats_main = DB_stats(customer_stats_original, "c_mktsegment")
mktsegment_stats_masked = DB_stats(customer_stats_masked, "c_mktsegment")
print(mktsegment_stats_main.dist.index)

In [ ]:
mktsegment_stats_masked.inverse_dist(inverse_suppress_segment)
mktsegment_stats_masked.inverse_histogram_bounds(inverse_suppress_segment)
mktsegment_stats_main.estimate_hist_freq(inverse_suppress_segment)
mktsegment_stats_masked.estimate_hist_freq(inverse_suppress_segment)

In [ ]:
estimate_rel_entropy(mktsegment_stats_main, mktsegment_stats_masked)

In [ ]:
stats_masked

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment'])
q = compute_prob_dist(customer_m1, ['c_mktsegment'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_suppress_segment)
#print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

In [ ]:
true_ent = 1.9408977646995316
est_ent = 8.279417859324237
np.abs(true_ent-est_ent)/true_ent

In [ ]:
size = 150000
# df.head()
stats_main.fillna(np.NaN, inplace=True)
stats_masked.fillna(np.NaN, inplace=True)
print(stats_masked.head())
stats_main["n_distinct"] = stats_main["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
stats_masked["n_distinct"] = stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [ ]:
# stats_masked.head()
mktsegment_stats_main = DB_stats(stats_main, "c_mktsegment")
mktsegment_stats_masked = DB_stats(stats_masked, "c_mktsegment")
mktsegment_stats_masked
mktsegment_stats_masked.inverse_dist(inverse_suppress_segment)
mktsegment_stats_masked.inverse_histogram_bounds(inverse_suppress_segment)
mktsegment_stats_main.estimate_hist_freq(inverse_suppress_segment)
mktsegment_stats_masked.estimate_hist_freq(inverse_suppress_segment)
print(mktsegment_stats_main.hist_apprx_freq)

In [ ]:
estimate_rel_entropy(mktsegment_stats_main, mktsegment_stats_masked)

In [ ]:
stats_masked

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment'])
q = compute_prob_dist(customer_m1, ['c_mktsegment'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_suppress_segment)
#print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

In [ ]:
true_ent = 1.9408977646995316
est_ent = 8.279417859324237
np.abs(true_ent-est_ent)/true_ent

## orders table

### order date noise

#### estimating relative entropy

In [ ]:
size = 150000
# df.head()
orders_stats_original.fillna(np.NaN, inplace=True)
orders_stats_masked_p1.fillna(np.NaN, inplace=True)

orders_stats_original["n_distinct"] = orders_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
orders_stats_masked_p1["n_distinct"] = orders_stats_masked_p2["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))

In [ ]:
o_orderdate_stats_original = DB_stats(orders_stats_original, "o_orderdate")

In [ ]:
o_orderdate_stats_p1 = DB_stats(orders_stats_masked_p1, "o_orderdate")

In [ ]:
# o_orderdate_stats_p1.perturbation()
o_orderdate_stats_original.perturbation()
# o_orderdate_stats_p1.inverse_dist(inverse_noise_date_SO("DAYS", 10))
# print(o_orderdate_stats_p1.unmsk_dist)
o_orderdate_stats_p1.inverse_histogram_bounds(inverse_noise_date_SO("DAYS", 10))
# for x in o_orderdate_stats_p1.unmsk_dist:
#     print (x)
# o_orderdate_stats_p1.unmsk_dist
print(o_orderdate_stats_p1.unmsk_dist)
print(o_orderdate_stats_original.unmsk_dist)
# for x in o_orderdate_stats_p1.unmsk_dist['1996-03-10'].keys():
#     print (type(x))


In [ ]:
entropy_o_noise_date_p1 = estimate_rel_entropy(o_orderdate_stats_original, o_orderdate_stats_p1)
print(entropy_o_noise_date_p1)

In [ ]:
print(all_dates)

In [ ]:
o_orderdate_stats_p1.estimate_hist_freq(all_dates)

In [ ]:
o_orderdate_stats_p1.hist_apprx_freq
print(o_orderdate_stats_p1.hist_apprx_freq)

In [ ]:
print(o_orderdate_stats_p1.unmsk_histogram_bounds)

In [ ]:
print(o_orderdate_stats_p1.unmsk_most_common_freqs)
print(o_orderdate_stats_p1.unmsk_most_common_vals)
print(o_orderdate_stats_p1.unmsk_dist)
print(o_orderdate_stats_p1.unmsk_histogram_bounds)

In [ ]:
estimate_rel_entropy(ordersdate_stats_main, ordersdate_stats_masked)

### order date generalization

#### estimating relative entropy

In [ ]:
size = 150000
# df.head()
orders_stats_original.fillna(np.NaN, inplace=True)
orders_stats_masked_p2.fillna(np.NaN, inplace=True)

orders_stats_original["n_distinct"] = orders_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
orders_stats_masked_p2["n_distinct"] = orders_stats_masked_p2["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))

In [ ]:
from datetime import datetime, timedelta

def generate_date_range(start_date_str, end_date_str):
    """
    Generate all dates between two given dates (inclusive).

    Args:
        start_date_str (str): The start date in "YYYY-MM-DD" format.
        end_date_str (str): The end date in "YYYY-MM-DD" format.

    Returns:
        list: A list of dates as strings in "YYYY-MM-DD" format.
    """
    try:
        # Convert string dates to datetime objects
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
        end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

        # Check if the start date is before or the same as the end date
        if start_date > end_date:
            raise ValueError("The start date must be before or the same as the end date.")

        # Generate the list of dates
        current_date = start_date
        date_list = []
        while current_date <= end_date:
            date_list.append(current_date.strftime("%Y-%m-%d"))
            current_date += timedelta(days=1)  # Move to the next day

        return date_list
    
    except ValueError as e:
        raise ValueError(f"Date error: {e}")

# Example usage
start_date = "1992-01-01"
end_date = "1998-08-02"
all_dates = generate_date_range(start_date, end_date)

In [ ]:
dates_list = set(list(orders_original["o_orderdate"].map(lambda x: x.strftime('%Y-%m-%d'))))
dates_list_alphabet = list(dates_list)
print(list(dates_list))

['1994-12-21', '1998-02-04', '1993-10-19', '1998-07-06', '1993-08-22', '1998-03-23', '1995-08-18', '1992-07-14', '1995-07-21', '1992-03-14', '1995-01-15', '1993-04-13', '1994-12-10', '1997-08-16', '1998-01-10', '1994-05-02', '1994-03-26', '1993-07-31', '1997-07-22', '1992-03-26', '1993-01-23', '1994-07-21', '1998-06-04', '1992-09-15', '1997-10-19', '1993-05-13', '1998-07-17', '1996-02-25', '1998-03-21', '1994-05-18', '1992-06-15', '1998-05-06', '1996-08-16', '1993-02-12', '1998-02-19', '1997-07-06', '1995-06-20', '1995-01-26', '1994-04-06', '1996-07-21', '1992-06-29', '1995-05-28', '1994-05-22', '1993-06-19', '1998-02-26', '1994-04-14', '1993-11-20', '1992-04-02', '1993-01-21', '1998-07-11', '1993-03-12', '1998-07-30', '1993-08-28', '1994-10-13', '1996-09-05', '1996-07-11', '1993-02-05', '1992-08-12', '1993-01-15', '1993-03-14', '1995-12-20', '1995-11-16', '1995-09-18', '1992-07-08', '1992-06-01', '1994-06-25', '1996-05-24', '1993-06-26', '1997-04-03', '1995-01-18', '1995-09-09', '1992

In [ ]:
ordersdate_stats_main = DB_stats(orders_stats_original, "o_orderdate")

In [ ]:
ordersdate_stats_masked = DB_stats(orders_stats_masked_p2, "o_orderdate")

In [ ]:
ordersdate_stats_masked.perturbation()

In [ ]:
# ordersdate_stats_masked.inverse_dist(inverse_generalize_date_SO("MONTH"))
ordersdate_stats_masked.inverse_histogram_bounds(inverse_generalize_date_SO("MONTH"))
# ordersdate_stats_main.estimate_hist_freq(all_dates)
# ordersdate_stats_masked.estimate_hist_freq(all_dates)
# print(ordersdate_stats_main.hist_apprx_freq)

In [ ]:
ordersdate_stats_masked.estimate_hist_freq(all_dates, True)
# ordersdate_stats_masked.addapt_freq_selectivity(0.5)

make graph from frequencies

In [ ]:
estimate_rel_entropy(ordersdate_stats_main, ordersdate_stats_masked)

-4.908223652361752

#### real relative entropy

In [ ]:
p = compute_prob_dist(orders_original, ['o_orderdate'])
q = compute_prob_dist(orders_masked_p1, ['o_orderdate'])

In [ ]:
print(p)

In [ ]:
print(q)

In [ ]:

new_q = specification_prob(q,inverse_generalize_date_SO("MONTH"))

In [ ]:
print(new_q)

In [ ]:
#print(new_q)
relative_entropy(p, new_q,  orders_masked_p2.size)

In [ ]:
vals = convert_to_array(get_stat(orders_stats_original, "o_orderdate", "most_common_vals"))
freqs = convert_to_array(get_stat(orders_stats_original, "o_orderdate", "most_common_freqs"), True)

p = pd.Series(freqs, index = vals)

vals = convert_to_array(get_stat(orders_stats_masked_p2, "o_orderdate", "most_common_vals"))
freqs = convert_to_array(get_stat(orders_stats_masked_p2, "o_orderdate", "most_common_freqs"), True)

q = pd.Series(freqs, index = vals)

new_q = specification_prob(q,inverse_generalize_date_SO("MONTH"))

relative_entropy(p, new_q, new_q.size)

In [ ]:
real_ent = 0.008263819318169217
est_ent = 0.028624210661386703
np.abs(real_ent-est_ent)/real_ent

#### estimating relative entropy

In [ ]:
size = 150000
# df.head()
orders_stats_original.fillna(np.NaN, inplace=True)
orders_stats_masked_p2.fillna(np.NaN, inplace=True)

orders_stats_original["n_distinct"] = orders_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
orders_stats_masked_p2["n_distinct"] = orders_stats_masked_p2["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))

In [ ]:
dates_list = set(list(orders_original["o_orderdate"].map(lambda x: x.strftime('%Y-%m-%d'))))
dates_list_alphabet = list(dates_list)
print(list(dates_list))

In [ ]:
ordersdate_stats_main = DB_stats(orders_stats_original, "o_orderdate")
print(ordersdate_stats_main.histogram_bounds)

In [ ]:
ordersdate_stats_masked = DB_stats(orders_stats_masked_p2, "o_orderdate")
print(ordersdate_stats_masked.histogram_bounds)

In [ ]:
ordersdate_stats_masked.perturbation
ordersdate_stats_masked.inverse_dist(inverse_generalize_date_SO("MONTH"))
ordersdate_stats_masked.inverse_histogram_bounds(inverse_generalize_date_SO("MONTH"))
ordersdate_stats_main.estimate_hist_freq(all_dates)
ordersdate_stats_masked.estimate_hist_freq(all_dates)
print(ordersdate_stats_main.hist_apprx_freq)

In [ ]:
ordersdate_stats_masked.estimate_hist_freq(all_dates, True)

In [ ]:
ordersdate_stats_masked.hist_apprx_freq
print(ordersdate_stats_masked.hist_apprx_freq)

In [ ]:
print(ordersdate_stats_masked.unmsk_histogram_bounds)

In [ ]:
print(ordersdate_stats_masked.unmsk_most_common_freqs)
print(ordersdate_stats_masked.unmsk_most_common_vals)
print(ordersdate_stats_masked.unmsk_dist)
print(ordersdate_stats_masked.unmsk_histogram_bounds)

In [ ]:
estimate_rel_entropy(ordersdate_stats_main, ordersdate_stats_masked)

-4.908223652361752

In [ ]:
p = compute_prob_dist(orders_original, ['o_orderdate'])
q = compute_prob_dist(orders_stats_masked_p2, ['o_orderdate'])

In [ ]:
print(p)

In [ ]:
print(q)

In [ ]:

new_q = specification_prob(q,inverse_generalize_date_SO("MONTH"))

In [ ]:
print(new_q)

In [ ]:
#print(new_q)
relative_entropy(p, new_q,  orders_masked.size)

In [ ]:
vals = convert_to_array(get_stat(orders_stats_original, "o_orderdate", "most_common_vals"))
freqs = convert_to_array(get_stat(orders_stats_original, "o_orderdate", "most_common_freqs"), True)

p = pd.Series(freqs, index = vals)

vals = convert_to_array(get_stat(orders_stats_masked, "o_orderdate", "most_common_vals"))
freqs = convert_to_array(get_stat(orders_stats_masked, "o_orderdate", "most_common_freqs"), True)

q = pd.Series(freqs, index = vals)

new_q = specification_prob(q,inverse_generalize_date_SO("MONTH"))

relative_entropy(p, new_q, new_q.size)